In [125]:
key_path = '../secrets/sheet-reader-key.json'
spreadsheet_id = '1VwC-GYUuEjsZr1DqGgeo8Wkv6_Ly6NZjAV07Zeik4ko'


In [26]:

import os

from apiclient import discovery
from google.oauth2 import service_account

scopes = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/drive.file', 'https://www.googleapis.com/auth/spreadsheets']
secret_file = os.path.join(os.getcwd(), key_path)


credentials = service_account.Credentials.from_service_account_file(secret_file, scopes=scopes)
service = discovery.build('sheets', 'v4', credentials=credentials)

In [177]:
def schedule_sheets_update(mode):
    if mode not in ['create','delete']:
        raise ValueError('Invalid mode')
    range_name = 'AIKATAULU!A1:D'
    template_id = '61864045'
    
    schedule = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name, valueRenderOption='UNFORMATTED_VALUE').execute()
    
    metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
    all_sheets = {p['properties']['title']:p['properties']['sheetId'] for p in metadata['sheets']}
    
    n_games = {}
    reqs = []
    
    update_cells = []
    for i, row in enumerate(schedule['values'][1:]):
        if len(row) == 4:
            
            game_id = f'{row[1]}{row[2]}{row[0]}'
            n_games[game_id] = n_games.get(game_id, 0) + 1

            name = f'{game_id}_{n_games[game_id]}'
            update_cells.append((name, True))

            if (mode == 'create'):
                if name not in all_sheets:
                    reqs.append({
                        "duplicateSheet": {
                            "sourceSheetId": template_id,
                            "newSheetName": name,
                        }
                    })                
                
            elif (mode=='delete' and name in all_sheets):
                reqs.append({
                    f"deleteSheet": {
                        "sheetId": all_sheets[name]
                    }
                })
        else:
            update_cells.append(('', False))

    if len(reqs) > 0:# Execute the batchUpdate request
        body = {"requests": reqs}
        response = service.spreadsheets().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body=body
        ).execute()
        
        metadata = service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        all_sheets = {p['properties']['title']:p['properties']['sheetId'] for p in metadata['sheets']}
    
   
    cell_updates = []
    if mode == 'create':
        url_base = f"""=HYPERLINK(\"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/edit"""
        names = [r['duplicateSheet']['newSheetName'] for r in reqs]
        for name, update in update_cells:
            if update:
                cell_updates.append([f"""{url_base}?gid={all_sheets[name]}#gid={all_sheets[name]}", "Gamelink")"""])
            else:
                cell_updates.append([""])

    if mode == 'delete':
        cell_updates = [[""] for i in schedule['values']]
    
    body = {
        'values': cell_updates
    }

    response = service.spreadsheets().values().update(
        spreadsheetId=spreadsheet_id,
        range="AIKATAULU!E2:E",  # Specify the range
        valueInputOption="USER_ENTERED",
        body=body
    ).execute()
    
    if mode == 'create':
        for i, row in enumerate(schedule['values'][1:]):
            if len(row) == 4:            
                game_id = f'{row[1]}{row[2]}{row[0]}'
                name = f'{game_id}_{n_games[game_id]}'
                data = [
                    {
                        'range': f'{name}!A2:A2',  # Specify the first range
                        'values': [[row[1]]]
                    },
                    {
                        'range': f'{name}!F2:F2',  # Specify the first range
                        'values': [[row[2]]]
                    }
                ]
        response = service.spreadsheets().values().batchUpdate(
            spreadsheetId=spreadsheet_id,
            body={'data':data, 'valueInputOption':'USER_ENTERED'}
        ).execute()
    
    
    move_requests = []
    for sheet_name in ['AIKATAULU','JOUKKUEET','NHL','TEMPLATE']:
        move_requests.append({
        "updateSheetProperties": {
            "properties": {
                "sheetId": all_sheets[sheet_name],
                "index": 1  # The desired index position (0-based)
            },
            "fields": "index"
        }})
    
    request_body_move = {
        "requests": [move_requests]
    }
    service.spreadsheets().batchUpdate(
        spreadsheetId=spreadsheet_id,
        body=request_body_move
    ).execute()

    return
    

In [191]:
import pandas as pd
def read_data(range_name):
    data = service.spreadsheets().values().get(spreadsheetId=spreadsheet_id, range=range_name, valueRenderOption='UNFORMATTED_VALUE').execute()
    return pd.DataFrame(data['values'][1:], columns=data['values'][0])

def read_players():
    range_name = 'NHL!A1:T'  
    return read_data(range_name)

def read_teams():
    range_name = 'JOUKKUEET!A1:B'  
    return read_data(range_name)

def read_schedule():
    range_name = 'AIKATAULU!A1:D'
    
    return read_data(range_name).dropna()

def read_players():
    range_name = 'NHL!A1:T'  
    return read_data(range_name)




In [240]:
def read_game_data():
    n_games = {}
    metadata_rows = []
    ranges = []
    for row in schedule.to_dict(orient='records'):
        game_id = f"{row['KOTI']}{row['VIERAS']}{row['RUNKOSARJA/PLAYOFFS']}"
        n_games[game_id] = n_games.get(game_id, 0) + 1

        name = f'{game_id}_{n_games[game_id]}'
        row['name'] = name
        ranges.append(f'{name}!A5:J30')
        metadata_rows.append(row)


    # Request to batch get values from multiple ranges
    response = service.spreadsheets().values().batchGet(
        spreadsheetId=spreadsheet_id,
        ranges=ranges,
        valueRenderOption='UNFORMATTED_VALUE'
    ).execute()

    goals = []
    for value_range, metadata in zip(response['valueRanges'], metadata_rows):
        metadata['game_state'] = value_range['values'][23][3]
        for row in value_range.get('values', [])[:20]:
            row_data = {
                'scoring_team':'KOTI',
                'scorer':row[0],
                'ass_1': row[1],
                'ass_2': row[2]
            }

            row_data.update(metadata)
            goals.append(row_data)

            row_data = {
                'scoring_team':'VIERAS',
                'scorer':row[0],
                'ass_1': row[1],
                'ass_2': row[2]
            }
            row_data.update(metadata)
            goals.append(row_data)
    goals = pd.DataFrame(goals)
    goals = goals[goals.scorer != '']
    return pd.DataFrame(metadata_rows), goals



In [235]:
# schedule_sheets_update('delete')
schedule_sheets_update('create')
# players = read_players()
# players
# teams = read_teams()
# schedule = read_schedule()


In [239]:
games, goals = read_game_data()
goals

,scoring_team,scorer,ass_1,ass_2,RUNKOSARJA/PLAYOFFS,KOTI,VIERAS,AREENA,name,game_state
